In [1]:
import requests
import time
import pprint
import pickle
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from unicodedata import normalize
from matplotlib import pyplot
import seaborn as sns
from sklearn.preprocessing import Imputer
%matplotlib
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)
from sklearn.preprocessing import LabelEncoder
from collections import Counter

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
tablin = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\procesamiento_1_2.csv', 
                    sep=';', engine="python", decimal=',', encoding="latin1")   
tablin= tablin.rename(columns = {'ID':'IDMIEMBRO'})

In [3]:
#Leemos la tabla 3
aumentos = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\3.AUMENTOS_train.txt', 
                    sep=';', engine="python", decimal=',', encoding="latin1")   

In [4]:
aumentos.columns = ['IDMIEMBRO', 'FECHA_aum', 'IDFAMILIA_aum', 'IDREGISTRO_aum', 'IDMEDIO_aum', 'IDCAMPANYA_aum']

In [5]:
aumentos = aumentos.loc[:,['IDMIEMBRO', 'IDREGISTRO_aum']]

In [6]:
#Calculamos cuantas veces el usuario ha aumentado sus donativos
aumentos= aumentos.groupby(['IDMIEMBRO']).sum()

In [7]:
aumentos= aumentos.reset_index()

In [12]:
tablin= tablin.merge(aumentos, how='outer')

In [13]:
#Leemos la tabla 5
bajas = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\5_bajas_final.csv', 
                    engine="python", decimal=',', encoding="latin1")   

In [15]:
bajas= bajas.loc[:,['IDMIEMBRO', 'baj_fech']]

In [16]:
#Nos quedamos con la última baja de cada usuario
bajas= bajas.groupby(['IDMIEMBRO']).max()
bajas= bajas.reset_index()

In [19]:
tablin= tablin.merge(bajas, how='outer')

In [20]:
#Leemos la tabla 6
enriq = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\6-enriquecimiento_20170201.txt', 
                    sep=';', engine="python", decimal=',', encoding="latin1")   

In [ ]:
#Eliminamos unas cuantas columnas que no creemos estrictamente necesarias
del enriq['ComunidadAutonoma']
del enriq['TramoEdad']
del enriq['Menores18']
del enriq['Entre18_45']
del enriq['T_Antiguedad']
del enriq['Superficie']
del enriq['T_Superficie']
del enriq['Superficie_NivelAsignacion']
del enriq['PoblacionINE']
del enriq ['Tasa2']
del enriq ['Valora']
del enriq ['T_Valora']
del enriq ['Antiguedad']
del enriq ['Habitat']
del enriq ['DistCapital']
del enriq['T_DistCapital']
del enriq ['T_TasaParo']
del enriq['OtrosElementos']
del enriq['T_Renta']

In [23]:
enriq=enriq.fillna(0)

In [24]:
#Categorizamos (de esta forma porque no nos leía el formato)
enriq= enriq.replace('6.NoInformado', '6')
enriq= enriq.replace('3.NoInformado', '3')
enriq= enriq.replace('NoInformado', '0')
enriq= enriq.replace('4.NoInformado', '4')
enriq= enriq.replace('0.NoInformado', '0')
enriq= enriq.replace('1.NoInformado', '1')
enriq= enriq.replace('2.NoInformado', '2')
enriq= enriq.replace('5.NoInformado', '5')
enriq= enriq.replace('7.NoInformado', '7')

In [25]:
enriq= enriq.replace('5.Medio Alto', '5')
enriq= enriq.replace('4.Medio', '4')
enriq= enriq.replace('3.Medio Bajo', '3')
enriq= enriq.replace('2.Bajo', '2')
enriq= enriq.replace('1.Muy Bajo', '1')
enriq= enriq.replace('6.Alto', '6')
enriq= enriq.replace('7.Muy Alto', '7')



In [26]:
labelencoder = LabelEncoder()
enriq['NivelEstudios'] = labelencoder.fit_transform(enriq['NivelEstudios'])

In [27]:
enriq= enriq.replace('CORUÃ\x91A, A', 'CORUÑA')
enriq= enriq.replace('ARABA/ALAVA', 'ALAVA')
enriq= enriq.replace('VALENCIA/VALENCIA', 'VALENCIA')
enriq= enriq.replace('CASTELLON/CASTELLO', 'CASTELLON')
enriq= enriq.replace('PALMAS, LAS', 'LASPALMAS')
enriq= enriq.replace('ALICANTE/ALACANT', 'ALICANTE')
enriq= enriq.replace('BALEARS, ILLES', 'ILLESBALEARS')

In [28]:
tablin= tablin.merge(enriq, how='outer')

In [29]:
#Leemos la tabla 8
inter = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\8.INTERACCIONES_TLMK_train.txt', 
                    sep=';', engine="python", decimal=',', encoding="latin1")   

In [31]:
inter= inter.loc[:,['IDMIEMBRO', 'FECHA', 'IDCANAL', 'IDMEDIO', 'ESTADO']]

In [32]:
inter=inter.fillna(0)

In [33]:
inter['FECHA'] = pd.to_datetime(inter['FECHA'], format= "%d/%m/%Y %H:%M:%S")

In [34]:
inter['year']= inter['FECHA'].dt.year
inter['month'] = inter['FECHA'].dt.month

In [35]:
inter['month'] = inter['month'].map("{:02}".format)
inter['FECHA']=(inter['year'].astype(str) + inter['month'].astype(str))

In [36]:
del inter['year']
del inter['month']

In [37]:
tablin.to_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\tablin_final', sep=';', encoding="latin1", decimal=',', index=False)

In [38]:
inter.to_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\interacciones', sep=';', encoding="latin1", decimal=',', index=False)